MTR sim agents pipeline for loading scenarios, predicted trajectories and doing evaluation

In [20]:
# Imports
import os
import tarfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tqdm
import json

from waymo_open_dataset.wdl_limited.sim_agents_metrics import metric_features
from waymo_open_dataset.wdl_limited.sim_agents_metrics import metrics

from waymo_open_dataset.protos import scenario_pb2
from waymo_open_dataset.protos import sim_agents_submission_pb2

from waymo_open_dataset.utils.sim_agents import submission_specs
from waymo_open_dataset.utils.sim_agents import test_utils as sim_agents_test_utils
from waymo_open_dataset.utils.sim_agents import visualizations
from waymo_open_dataset.utils import trajectory_utils

# Set matplotlib to jshtml so animations work with colab.
from matplotlib import rc
rc('animation', html='jshtml')


Loading data and making the dataset

In [21]:
DATASET_FOLDER = '/home/mbultc/womd1_1/'
TRAIN_FILES = os.path.join(DATASET_FOLDER, 'training/*')
VALIDATION_FILES = os.path.join(DATASET_FOLDER, 'mini_womd1_1/validation/*')
TEST_FILES = os.path.join(DATASET_FOLDER, 'testing/*')

# Define the dataset from the TFRecords.
filenames = tf.io.matching_files(VALIDATION_FILES)
print(f'Number of tfrecord files read: {filenames.shape}')
dataset = tf.data.TFRecordDataset(filenames)
# Since these are raw Scenario protos, we need to parse them in eager mode.
dataset_iterator = dataset.as_numpy_iterator()
#print(f'Number of scenarios in dataset iterator: {dataset.__len__()}')

# Load data from the JSON file
with open("/home/mbultc/PycharmProjects/MTR/data/saved_tracks_json/first_results.json", "r") as json_file:
    loaded_data = json.load(json_file)

# Retrieve tensors
base64_rollouts_tensors = loaded_data["rollouts_tensors"]
base64_object_ids_tensors = loaded_data["object_ids_tensors"]
scenario_id_list_json = loaded_data["scenario_ids"]

# Convert base64-encoded strings back to byte strings
serialized_rollouts_tensors = [tf.io.decode_base64(tensor.encode('utf-8')) for tensor in base64_rollouts_tensors]
serialized_object_ids_tensors = [tf.io.decode_base64(tensor.encode('utf-8')) for tensor in base64_object_ids_tensors]

# Deserialize tensors
rollouts_tensors_list = [tf.io.parse_tensor(tensor, out_type=tf.float32) for tensor in serialized_rollouts_tensors]
object_ids_list = [tf.io.parse_tensor(tensor, out_type=tf.int32) for tensor in serialized_object_ids_tensors]

print(f'Number of predictions: {len(object_ids_list)}')

Number of tfrecord files read: (1,)
Number of predictions: 4


Define some useful functions

In [22]:
def joint_scene_from_states(
    states: tf.Tensor, object_ids: tf.Tensor
    ) -> sim_agents_submission_pb2.JointScene:
  # States shape: (num_objects, num_steps, 4).
  # Objects IDs shape: (num_objects,).
  states = states.numpy()
  simulated_trajectories = []
  for i_object in range(len(object_ids)):
    simulated_trajectories.append(sim_agents_submission_pb2.SimulatedTrajectory(
        center_x=states[i_object, :, 0], center_y=states[i_object, :, 1],
        center_z=states[i_object, :, 2], heading=states[i_object, :, 3],
        object_id=object_ids[i_object]
    ))
  return sim_agents_submission_pb2.JointScene(
      simulated_trajectories=simulated_trajectories)

def scenario_rollouts_from_states(
    scenario: scenario_pb2.Scenario,
    states: tf.Tensor, object_ids: tf.Tensor
    ) -> sim_agents_submission_pb2.ScenarioRollouts:
  # States shape: (num_rollouts, num_objects, num_steps, 4).
  # Objects IDs shape: (num_objects,).
  joint_scenes = []
  for i_rollout in range(states.shape[0]):
    joint_scenes.append(joint_scene_from_states(states[i_rollout], object_ids))
  return sim_agents_submission_pb2.ScenarioRollouts(
      # Note: remember to include the Scenario ID in the proto message.
      joint_scenes=joint_scenes, scenario_id=scenario.scenario_id)

from google.protobuf import text_format
from waymo_open_dataset.protos import sim_agents_metrics_pb2

def load_metrics_config() -> sim_agents_metrics_pb2.SimAgentMetricsConfig:
    """Loads the `SimAgentMetricsConfig` used for the challenge."""
    # pylint: disable=line-too-long
    # pyformat: disable
    config_path = '/home/mbultc/PycharmProjects/Thesis/waymo-open-dataset/src/waymo_open_dataset/wdl_limited/sim_agents_metrics/challenge_config.textproto'
    with open(config_path, 'r') as f:
        config = sim_agents_metrics_pb2.SimAgentMetricsConfig()
        text_format.Parse(f.read(), config)
    return config



Iterate over scenarios and predicted trajectories and do evaluation

In [23]:
counter = 0
for bytes_example in dataset_iterator:
    scenario = scenario_pb2.Scenario.FromString(bytes_example)
    assert scenario.scenario_id == scenario_id_list_json[counter], "Mismatch between scenario ids"
    print(f'Doing evaluation for scenario: {scenario.scenario_id}')
    
    simulated_states = rollouts_tensors_list[counter]
    object_ids = object_ids_list[counter]
    
    scenario_rollouts = scenario_rollouts_from_states(scenario, simulated_states, object_ids)
    # As before, we can validate the message we just generate.
    submission_specs.validate_scenario_rollouts(scenario_rollouts, scenario)

    # Load the test configuration.
    config = load_metrics_config()

    scenario_metrics = metrics.compute_scenario_metrics_for_bundle(config, scenario, scenario_rollouts)
    print(scenario_metrics)
    
    counter += 1
    if counter >= len(rollouts_tensors_list):
        break

Doing evaluation for scenario: 637f20cafde22ff8


2024-04-18 20:34:48.272168: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:219] failed to create cublas handle: cublas error
2024-04-18 20:34:48.272280: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:221] Failure to initialize cublas may be due to OOM (cublas needs some free memory when you initialize it, and your deep-learning framework may have preallocated more than its fair share), or may be because this binary was not built with support for the GPU in your machine.
2024-04-18 20:34:48.272298: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at matmul_op_impl.h:629 : INTERNAL: Attempting to perform BLAS operation using StreamExecutor without BLAS support


InternalError: {{function_node __wrapped__Einsum_N_2_device_/job:localhost/replica:0/task:0/device:GPU:0}} Attempting to perform BLAS operation using StreamExecutor without BLAS support [Op:Einsum]